# Sequences and RNNs
## Intro to Sequential Data and History of RNN
### Sequential Data
#### Time series
#### Text processing
##### Applications
- Classification (Sentiment analysis, SPAM detection)
- Translation
- Captioning
- Dialog
### History of RNN
#### Early stage 90s
- Schmidhuber, Hochreiter, Bengio identify vanishing / exploding gradient problem
- Hochreiter invents LSTM 1997

#### 2013 2014 All Hell Breaks loose
#### 2015 Attention
#### 2017 RNNs are more and more declared obsolete
### What constitutes an RNN
- Diagrams of possible setups, gathered from places
- Diagrams of possible internal processing
$$h_{t+1} = \rho(W_{hh}h_t + W_{xh}x_t)$$


## Intro to Coding RNNs in Pytorch
The goal of this part is to become familiar with the code building blocks that can be assembled into an RNN. From architecture specification, to training, from data preparation to evaluation nothing is left out.

In a first step we identify common aspects between many RNNs and will bring them to life using some small examples which nevertheless show interesting properties.

In a second step we shall work on bigger architectures for text analysis.

### Architecture Blocks
We will identify building blocks in typical pytorch RNN architectures that can be used to quickly set up an RNN and evaluate how they differ from other neural network architectures.

#### `__init__` and `build` methods
The `__init__` and `build` methods take care of storing parameters and setting up the data structures for training. While `__init__` should only store parameters (as per the useful `scikit-learn` conventions), `build` creates all the trainable `Parameter`s:

```python
def __init__(self, input_dim, hidden_dim, output_dim):
    super(ClassName, self).__init__()  # Pytorch requires this superclass init
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.output_dim = output_dim

def build(self):
    # Here we set up some parameters, for example:
    self.input_to_hidden = torch.nn.Linear(self.input_dim, self.hidden_dim)
    self.hidden_to_hidden = torch.nn.Linear(self.hidden_dim, self.hidden_dim)
    self.hidden_to_output = torch.nn.Linear(self.hidden_dim, self.output_dim)
```

In this example, our build method creates three affine mappings, one from input to hidden, one from hidden to hidden, and one from hidden to output. These mappings are trainable and will be used by the RNN step logic

#### The `step` function
The `step` function is a point in which RNNs differ from many other neural network architectures. In the step function, the current hidden state is manipulated by a specific operation, which may or may not involve using a data input and yielding an output.

The step function is an important unit of core functionality of the RNN.

Here we implement the (*fully linear*) step
$$h_t = W_{hh}h_{t-1} + W_{xh}x_t$$
$$y_t = W_{hy}h_t$$


```python
def step(self, x_in, hidden):
    # Eg use the input and a hidden state to create a new hidden state and an output
    new_hidden_from_hidden = self.hidden_to_hidden(hidden) 
    new_hidden_from_input = self.input_to_hidden(x_in)
    new_hidden = new_hidden_from_hidden + new_hidden_from_input
    y_out = self.hidden_to_output(new_hidden)
    return y_out, new_hidden
```

#### The `forward` function
This is the central function of a neural network module class. It should be able to take an input sample (or ideally a batch of samples) and output something that can be evaluated with a loss function.

In the case of RNNs, the input sample is a sequence of something.

The type of output can vary. For example, it could be a point-to point output predicting the next step.

```python
def forward(self, x):
    # We need to assume a shape, e.g. (input_features, time)
    n_features, n_time = x.shape
    # initialize a hidden state:
    hidden = initial_hidden_state
    
    outputs = []
    
    # Loop over input data and call step
    for i in range(n_time):
        # take time point
        xx = x[:, i]
        # do step
        out, hidden = self.step(xx, hidden)
        outputs.append(out)
    
    return outputs
```

#### The `train` and `test` functions
The train function resembles those for most other neural networks. We usually split it into a function that trains one epoch of a data set and function that iterates over epochs.

The function iterating over one train epoch with gradient updates takes a `model`, a `dataset`, an evalutation `criterion`, and an `optimizer`.

```python
def train_epoch(model, dataset, criterion, optimizer):
    loss_values = []
    for x, y in dataset:
        prediction = model(x)
        loss = criterion(prediction, y)
        loss_values.append(loss.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return np.mean(loss_values)
```

The function for evaluating on a test set looks quite similar
```python
def test(model, dataset, criterion):
    loss_values = []
    with torch.no_grad(): # switching of gradients makes things faster
        for x, y in dataset:
            prediction = model(x)
            loss = criterion(prediction, y)
            loss_values.append(loss.detach().cpu().numpy())
    return np.mean(loss_values)
```


The function iterating over epochs could look like this:
```python
def train_mse_adam(model, dataset, n_epochs=10, test_dataset=None):
    criterion = torch.nn.MSELoss() # We define the loss criterion here
    optimizer = torch.optim.Adam(model.parameters())
    
    train_losses = []
    test_losses = []
    for i in range(n_epochs):
        train_loss = train_epoch(model, dataset, criterion, optimizer)
        train_losses.append(train_loss)
        if test_dataset is not None:
            test_loss = test(mode, test_dataset, criterion)
            test_losses.append(test_loss)
    return train_losses, test_losses
```

#### In some cases, a `generate` function is useful
Imagine you have trained an RNN to predict the next item of a sequence, but now you would like to use it to generate a new sequence from a starting point by predicting the next point and feeding that back in as input.

Here is a way we can do this using the `step` function:
```python
def generate(rnn, starting_point, n):
    # generate n time points by plugging output back into input
    
    hidden = self.initial_hidden_state
    x = starting_point
    outputs = []
    for i in range(n):
        x, hidden = rnn.step(x, hidden)
        outputs.append(x)
    return outputs
```

### Small RNNs
With all this information about how to structure an RNN with pytorch, let's dive into some examples!

In this section, we will go through a fairly detailed series of small neural networks, showing examples of the different types of RNN architectures, adding complexity along the way and getting familiar with the pytorch functionatities for RNN.

Let's start by importing some of the necessary packages

In [ ]:
import numpy as np
import torch
import torch.utils.data

import matplotlib.pyplot as plt
%matplotlib inline

#### A linear RNN that models sinusoids - simple dynamical system
In this example we will generate a synthetic sinusoidal signal and see if we can model it with a very simple linear RNN. 

This is basically the simplest RNN possible and will get us started with the necessary scaffolding for setting up RNN training.

It is not an uninteresting example though: Through experiments we will learn a few things about linear dynamical systems and RNN computational capacity.

Architecture-wise we will be looking at two things:
- A continuous-output network which outputs a value at every time step: The prediction of the next value
- A short-circuiting of this process to create a generator

As a reminder, this corresponds to the two following settings:

<img src="images/rnn_many_to_many.png" width="500"></img>
<img src="images/rnn_seq_to_seq.png" width="450"></img>




Let's take a look at some sinusoidal data. In the next cell there is a function that can generate sinusoids of different frequencies.

In [ ]:
def generate_sin(n_samples, sin_frequencies=(1.,), weights=None, 
                 n_time_steps=100, sample_frequency=0.1, noise_level=0.):
    time_vals = np.arange(0, n_time_steps) * sample_frequency
    phase_offsets = np.random.rand(n_samples, len(sin_frequencies)) * 2 * np.pi
    if weights is None:
        weights = np.ones(len(sin_frequencies)) / len(sin_frequencies)
    
    sin_frequencies = np.asarray(sin_frequencies)
    pure_sinusoids = np.sin(2 * np.pi * time_vals[:, np.newaxis] * sin_frequencies
                           + phase_offsets[:, np.newaxis])
    complex_sinusoids = pure_sinusoids.dot(weights)
    return complex_sinusoids

We can test the function by generating a few sinusoids of one frequency.

In [ ]:
sinusoids = generate_sin(10, sin_frequencies=(1.,))
plt.figure(figsize=(10, 1))
plt.plot(sinusoids.T)
pass

##### Set up the linear RNN
To set up the linear RNN, we can almost copy and paste the functions from the introductory description above.

In [ ]:
class LinRNN(torch.nn.Module):
    
    def __init__(self, n_input=1, n_hidden=1, n_output=1):
        super(LinRNN, self).__init__()
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        self.build()
    
    def build(self):
        self.input_to_hidden = torch.nn.Linear(self.n_input, self.n_hidden)
        self.hidden_to_hidden = torch.nn.Linear(self.n_hidden, self.n_hidden)
        self.hidden_to_output = torch.nn.Linear(self.n_hidden, self.n_output)
        
        self.initial_hidden_state = torch.nn.Parameter(torch.randn(self.n_hidden) * 0.01)
    
    def step(self, x, hidden):
        # assume x.shape = (batch, features)
        new_hidden = self.input_to_hidden(x) + self.hidden_to_hidden(hidden)
        output = self.hidden_to_output(new_hidden)
        return output, new_hidden
    
    def forward(self, x):
        # assume x.shape = (batch, time, features)
        B, T, F = x.shape
        
        hidden = self.initial_hidden_state
        outputs = torch.empty((B, T, self.n_output))
        for t in range(T):
            xx = x[:, t, :]
            out, hidden = self.step(xx, hidden)
            outputs[:, t, :] = out
        return outputs

Similarly, the auxiliary functions for training and evaluating the network correspond to the prototypes from above.

We have a function that can perform one run through the full dataset:

In [ ]:
def train_epoch(model, dataset, criterion, optimizer):
    losses = []
    for x, y in dataset:
        p = model(x)
        loss = criterion(p, y)
        losses.append(loss.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return np.array(losses)

We have a function that can perform a test run. The difference to above is that no gradients are required.

In [ ]:
def test(model, dataset, criterion):
    losses = []
    with torch.no_grad():
        for x, y in dataset:
            p = model(x)
            loss = criterion(p, y)
            losses.append(loss.detach().cpu().numpy())
    return np.array(losses)

Combining the two above, we have the training function, which repeats the training process for a number of epochs.

In [ ]:
def train_mse_adam(model, dataset, test_dataset=None, n_epochs=2):
    train_losses = []
    test_losses = []
    
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    for e in range(n_epochs):
        train_loss = train_epoch(model, dataset, criterion, optimizer)
        train_losses.append(train_loss)
        if test_dataset is not None:
            test_loss = test(model, test_dataset, criterion)
            test_losses.append(test_loss)
    if test_dataset is not None:
        return np.array(train_losses), np.array(test_losses)
    return np.array(train_losses)

Finally, we have a little helper that can glue two datasets together. This is useful to create input and target sequences, because the target is just to input shifted by 1 (we are trying to predict the next step!)

In [ ]:
class ZipDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets
    def __len__(self):
        return min(len(dataset) for dataset in self.datasets)
    def __getitem__(self, i):
        return tuple(dataset[i] for dataset in self.datasets)

We can now generate some train and test data, which we convert to torch arrays. We add an axis to represent a 1D input feature space

In [ ]:
train_data = torch.from_numpy(generate_sin(1000)[:, :, np.newaxis].astype('float32'))
test_data = torch.from_numpy(generate_sin(100)[:, :, np.newaxis].astype('float32'))

We create our input and target data by simply shifting the train data on the time axis, zipping the two together using our zipper helper, and using the torch `DataLoader` class to create an object that yields batches.

In [ ]:
train_dataloader = torch.utils.data.DataLoader(ZipDataset(train_data[:, :-1], train_data[:, 1:]),
                                               batch_size=2, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(ZipDataset(test_data[:, :-1], test_data[:, 1:]),
                                              batch_size=10)

Now we can create an RNN. Remember the constructor from above? It is `LinRNN(n_input=1, n_hidden=1, n_output=1)`. Let's create one that takes 1 input value and yields 1 output value and uses 1 hidden unit. These are exactly the default values from above, but it doesn't hurt to specify them here:

In [ ]:
lrnn = LinRNN(n_hidden=1, n_input=1, n_output=1)

Now let's train it!

In [ ]:
train_loss, test_loss = train_mse_adam(lrnn, train_dataloader, test_dataloader, n_epochs=5)

The training process outputs both train and test losses in their raw, batched form. Their shapes are `(n_epochs, n_batches_per_epoch)`

In [ ]:
train_loss.shape, test_loss.shape

People often average the loss over batches to obtain just one value per epoch. In our case it might be instructive  to take a look at the full extent of the loss batches, because the optimization is actually quite fast.

Let's take a look at the training loss curves. We can plot each epoch separately by plotting the transpose of the train loss array (`matplotlib` plots matrix columns as lines):

In [ ]:
plt.plot(train_loss.T)
pass

But we can also concatenate all the losses and get a more global picture:

In [ ]:
plt.figure(figsize=(20, 2))
plt.plot(np.concatenate(train_loss))
pass

Next, we can take a look at the performance on the test set in basically the same way. Note that the test set batches were bigger, and there were fewer points in total, so the whole curve has much fewer points

In [ ]:
plt.figure(figsize=(20, 2))
plt.plot(np.concatenate(test_loss))
pass

The test loss gets evaluated after every training batch. This is why we see such a steppy function: The performance is just much better after one full train epoch has passed

Next we would like to take a look at some individual predictions. We can visually evaluate how well it does at the task that it was trained on - predict the next element of the sequence.

Let's take a sample from the test data for that

In [ ]:
test_sample = test_data[0]

And plot it, just to be sure:

In [ ]:
plt.figure(figsize=(20, 2))
plt.plot(test_sample.numpy())

Now we push it through the RNN:

In [ ]:
test_sample_next = lrnn(test_sample[np.newaxis]).detach().cpu().numpy().ravel()

And we can take a look at the prediction compared to the sample

In [ ]:
plt.figure(figsize=(20, 2))
plt.plot(test_sample_next, label='next')
plt.plot(test_sample.numpy().ravel(), label='current')
plt.legend()
pass

- Wait ..., so the blue curve (*"next"*) is **in front of** the orange one? Why is this?
- What else do you observe? How could one address this?

##### Evaluation through generation

Since this model predicts the next item of the sequence, we have another interesting check: We can use the output of one time point as the input for the next! 

This corresponds to the second diagram above, where we reuse the network predictions as input. Note that this is a harder evaluation: Not only was the network not trained to do this task, but also "looking into itself" might lead to some weird recurrent behavior.

Let's check to see what happens when we do that. We start with an initial time point, grabbed from the test sample, and the initial hidden state of the RNN.

In [ ]:
x = test_sample[np.newaxis, 0, :]
hidden = lrnn.initial_hidden_state

With these two value we can feed the `step` function of our neural network. This will give an output and a new hidden state:

In [ ]:
x, lrnn.step(x, hidden)

We can use this in a loop: Feed `x` and the current `hidden` into the step function, and play the output back into `x` and `hidden`. Keep track of `x` and do this a few times to generate a sequence.

In [ ]:
generated = [x]
for i in range(100):
    x, hidden = lrnn.step(x, hidden)
    generated.append(x.detach().cpu().numpy())
generated = np.array(generated)


In [ ]:
plt.plot(generated)

Huh, so that worked less well. Why is that?

Exercise:
- play around with the number of hidden states to see if you can fix this
- if you manage to fix it: why do you think it works now?
- try more complex sinusoids by adding frequencies to `generate_sin`

#### A tanh one that can replay or reverse a short sequence of a small alphabet
Now that we have trained a linear RNN on continuous data, let's move on to something else: an RNN that works on symbols. Since linear networks probably won't work for this, let's directly make a nonlinear one using a `tanh` nonlinearity.

Also, note that we have to come up with a way of knowing which symbol to output, and also how to input symbols into RNNs.

The answer to this is *one-hot-encoding*, whereby every symbol is embedded into a space of dimensionality corresponding to the total number of symbols. Each symbol is assigned to one of the cartesian dimensions in that space and is thus represented by a vector of mostly zeros, except for one 1 in one of the entries

<img src="images/onehot.png"></img>

We will write a network to perform the so-called "copy task", that is: parse an input sequence with an encoder RNN into a hidden state, and then decode the original sequence or its reversed order in the output.

This task will permit us to understand some properties of simple symbol RNNs and the technicalities of training them. It will provide a starting point for incorporating built-in RNN modules to make code more concise, standardized, and faster.

It's architecture is of the following type
<img src="images/rnn_input_to_latent_output_from_latent.png"></img>

E.g.: **the &nbsp;&nbsp;&nbsp;&nbsp;cat &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;sat  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; on &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  the &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   mat** --> **the &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  cat  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; sat &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  on  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; the  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; mat**

##### Let's start by creating some data to get an idea

We start with an alphabet, which we just define as a list of symbols. Let's use a small one

In [ ]:
alphabet = np.array(list('abc'))
alphabet

Using this alphabet, we can generate random symbol sequences.

In [ ]:
n_sequences = 1000
sequence_length = 10
sequences = np.random.choice(alphabet, (n_sequences, sequence_length))
sequences[0:2]

(side note: the `dtype` of this `numpy` array, `<U1` means that this array contains entries of unicode strings of length less than or equal to 1)

In order to be able to give these sequences to an RNN, we need to put them into vector format. This is done by *One-Hot* encoding as mentioned before.

In order to create a one-hot encoded array quickly and easily, we can use the *implicit broadcasting* of numpy: We add an axis to our 2D array `sequences`, making it a 3D array, where the last entry of the shape is `1`. We then check each entry of the sequences for equality with each entry of the alphabet. This will broadcast the last axis to be of the size of the alphabet.
After this is done, we convert the output to `float32`

In [ ]:
sequences_one_hot = (sequences[..., np.newaxis] == alphabet).astype('float32')

It might be useful to take a moment to break every step down:

- Add an axis to `sequences`, check shape
- Add two empty axes to `alphabet`, check shape
- Perform equality
- Use the fact that implicit broadcasting *prepends as many 1s as needed* to the shorter-shaped array and drop the adding of axes to `alphabet`
- convert the binary output to floats

Lets check the first two one-hot-encoded sequences

In [ ]:
sequences_one_hot[:2]

##### Create an encoder RNN
We now set about building the first half of the encoder-decoder network, the encoder.

We create an RNN just like the linear one above, with two modifications:
- we add a nonlinearity, then hyperbolic tangent;
- we make the RNN output only the last hidden state.

The first point is accomplished by using `torch.tanh` in the `step` function. The second point is done by simply returning only `hidden` from `forward` after the for loop ends.

In [ ]:
class SeqEncoder(torch.nn.Module):
    def __init__(self, n_input, n_hidden):
        super(SeqEncoder, self).__init__()
        self.n_input = n_input
        self.n_hidden = n_hidden
        
        self.build()
    
    def build(self):
        self.input_to_hidden = torch.nn.Linear(self.n_input, self.n_hidden)
        self.hidden_to_hidden = torch.nn.Linear(self.n_hidden, self.n_hidden)
        
        self.initial_hidden_state = torch.nn.Parameter(torch.randn(self.n_hidden) * 0.01)

    def step(self, x, hidden):
        new_hidden = torch.tanh(self.input_to_hidden(x) + self.hidden_to_hidden(hidden))
        return new_hidden
    
    def forward(self, x):
        # assume x.shape = batch, time, features
        batch_size, time_length, n_features = x.shape
        hidden = self.initial_hidden_state * torch.ones(batch_size, 1).to(self.initial_hidden_state.device)
        
        for i in range(time_length):
            xx = x[:, i]
            hidden = self.step(xx, hidden)
        return hidden

Let's give it a quick try, by instantiating an encoder with 3 input dimensions (corresponding to alphabet length) and 10 hidden units. Any input sequences should end up encoded as a 10-dimensional vector.

In [ ]:
seq_encoder = SeqEncoder(n_input=3, n_hidden=10)

We can use some of our one-hot-encoded sequences from before to check this. Let's input two of them. We should get an output of shape `(2, 10)`.

In [ ]:
encoded = seq_encoder.forward(torch.from_numpy(sequences_one_hot[:2]))
encoded.shape, encoded

As expected, this yields two 10-dimensional hidden-state vectors!

##### Create a decoder RNN

The second half of an encoder/decoder network is the decoder. It is an RNN that takes an initial hidden state, but no inputs, and just iterates hidden state and output using its weights.

To create this, we modify the step function from the above encoder to only take a hidden state as input (no input sequence is used) and return a new hidden state and an output.

The forward function iterates this step over hidden states and gathers all the outputs, which it then returns. It is necessary to add an indication of how long we want the output sequence to be, so we put and argument `n` for that.

In [ ]:
class SeqDecoder(torch.nn.Module):
    def __init__(self, n_hidden, n_output):
        super(SeqDecoder, self).__init__()
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        self.build()
    
    def build(self):
        self.hidden_to_hidden = torch.nn.Linear(self.n_hidden, self.n_hidden)
        self.hidden_to_output = torch.nn.Linear(self.n_hidden, self.n_output)
    
    def step(self, hidden):
        new_hidden = torch.tanh(self.hidden_to_hidden(hidden))
        output = self.hidden_to_output(new_hidden)
        return output, new_hidden
    
    def forward(self, initial_hidden, n):
        batch_size, n_hidden = initial_hidden.shape
        
        hidden = initial_hidden
        outputs = torch.zeros(batch_size, n, self.n_output)
        for i in range(n):
            output, hidden = self.step(hidden)
            outputs[:, i] = output
        
        return outputs
        
        

Let's give this decoder a try immediately. We can create one with 10 hidden units and 3 output units. This corresponds to the hidden state of before and the alphabet length of our sequence.

In [ ]:
seq_decoder = SeqDecoder(10, 3)

Remember the input to the encoder gave us a shape of `(2, 10)` for 2 sequences and their hidden state? Let's use something of that shape for our decoder and ask for a sequence of length 5 as output.

In [ ]:
output = seq_decoder(torch.randn(2, 10), 5)
output.shape, output

As expected, the output is of shape `(2, 5, 3)` for 2 examples, length 5 and alphabet length 3.

We can also try decoding the variable `encoded` from before:

In [ ]:
seq_decoder(encoded, 6)

And that just concluded a full step of encoding and decoding a sequence!

##### Now let's create an encoder-decoder RNN combining the two
The next step is to create an object that does both encoding and decoding, so that we can train it with sequence data. We will thus create a small pytorch module containing an encoder and a decoder. All it does is book-keeping: Given an input dimension (alphabet length) and a hidden dimension, it can build an encoder and a decoder. The `forward` function takes an input sequence `x`, encodes it, and then decodes a sequences of exactly the same length.

In [ ]:
class SeqEncoderDecoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SeqEncoderDecoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        self.build()
    
    def build(self):
        self.encoder = SeqEncoder(self.input_dim, self.hidden_dim)
        self.decoder = SeqDecoder(self.hidden_dim, self.input_dim)
    
    def forward(self, x):
        _, time_length, _ = x.shape
        hidden = self.encoder(x)
        output = self.decoder(hidden, time_length)
        return output

Let's push some data through it. The input shape should correspond with the output shape.

We instantiate and encoder-decoder with 3 input units and 10 hidden units

In [ ]:
seq_encoder_decoder = SeqEncoderDecoder(3, 10)

And then we put our two initial sequences through it.

In [ ]:
encoded_decoded = seq_encoder_decoder(torch.from_numpy(sequences_one_hot[:2]))
encoded_decoded

Both input and output have the same shape, and can now be compared to one another

In [ ]:
encoded_decoded.shape, sequences_one_hot[:2].shape

##### Defining a loss function to optimize

How do we go about defining a loss function that makes our RNN understand that its output should be close to its input, for all inputs?

We could use MSE as for the sinusoid, but recall that our inputs and our outputs are supposed to reflect categorical variables. An appropriate differentiable loss for categorical variables is *cross entropy*, which is the loglikelihood of classification problems. It is defined as
$$\mathbb E_p[\log q]$$
for two distributions $p$ and $q$ over categories. In our case we are dealing with an estimate of this expectation, where the variable $p$ represents the one-hot-encoded "sure" probability of a certain class and $q$ represents the more uncertain prediction, which is the softmax of our neural network outputs:
$$q_j(y) = \textrm{softmax}(y)_j = \frac{\exp y_j}{\sum_k\exp y_k}$$

The estimator of the loglikelihood estimated from samples indexed with $i$ writes as
$$\frac{1}{N}\sum_{i=1}^N \sum_{j=1}^M p^i_j\log q^i_j,$$
where $M$ is the number of targets.

Given that $p_i$ is just a one-hot code indicating which label $j$ was active, by calling the label $l_i$, this can be rewritten as
$$\frac{1}{N}\sum_{i=1}^N \log q^i_{l_j}.$$

Basically this is indexing the output variable with the label. Maximizing this quantity will make the label more probable.

It is also in this indexing way that `pytorch` built-in loss functions are written out.

To use these mechanisms, we need our target variable (the one we compare to) to be a number indicating which dimension was correct.

Let's create that here by restarting from scratch with our sequences, using the `np.unique` function:

In [ ]:
alphabet, isequences = np.unique(sequences, return_inverse=True)

By using `return_inverse=True`, we not only get the (sorted) alphabet used, but also full sequences of indices into that alphabet representing which of the symbols was used in each sequence.

In [ ]:
alphabet, isequences

We could get the sequences back by fancy-indexing one with the other:

In [ ]:
alphabet[isequences]

However, note that `np.unique` destroys the shape of the input:

In [ ]:
alphabet[isequences].shape, sequences.shape

So we have to set it back:

In [ ]:
isequences.shape = sequences.shape

As above, we can now one-hot-encode these sequences to obtain neural network input

In [ ]:
isequences_one_hot = (isequences[..., np.newaxis] == np.arange(len(alphabet))).astype('float32')
isequences_one_hot[:2], isequences[:2]

But we also have the exact index sequence, which we can use for the loss:

We are sure we have a correspondence between `isequences` and `isequences_one_hot`.

This being fixed, we can now look at the `CrossEntropyLoss` object of `pytorch` and figure out what inputs it requires

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

Let's take a look at its docstring

In [ ]:
# criterion?

And let's try it out!

we encode and decode two of our one-hot-encoded sequences and get one-hot predictions out:

In [ ]:
encoded_decoded = seq_encoder_decoder(torch.from_numpy(isequences_one_hot[:2]))

Let's check that

In [ ]:
isequences_one_hot.shape, encoded_decoded.shape, isequences_one_hot[:2], encoded_decoded[:2]

Now let's try the criterion on our predictions, and the indices

In [ ]:
#criterion(encoded_decoded, torch.from_numpy(isequences[:2]))

The above errors because of a shape problem (look at debugger!)

Below you see the shapes of `encoded_decoded` and the target `isequences[:2]`. It seems the above criterion is iterating over the wrong axis (it seems to iterate over `axis=1` since the shape it extracts is `2, 3`).

In [ ]:
encoded_decoded.shape, isequences[:2].shape

Take another look at the docstring of the function to figure out what might be wrong!

**Exercise:** What do we need to change to make this code work?

If we permute the axes, we should make the criterion understand our data better:

In [ ]:
encoded_decoded.permute(0, 2, 1).shape

In [ ]:
criterion(encoded_decoded.permute(0, 2, 1), torch.from_numpy(isequences[:2]))

After transposing the output, we get a negative loglikelihood loss which we can use in gradient descent!

##### Creating a dataset iterator

Now that we have obtained a working evaluation criterion, next we need to create a dataset that consists of one-hot encoded sequences as input, and the corresponding integer sequences as output.

Let's inherit from the `torch.utils.data.Dataset` object, have it store some arbitrary sequences, and incorporate functionality that a) creates the sequences of unique integers b) the corresponding one-hot-encoded vectors

In [ ]:
class SeqDataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
        
        self.build()
    
    def build(self):
        self.alphabet, self.isequences = np.unique(self.sequences, return_inverse=True)
        self.isequences.shape = self.sequences.shape
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, i):
        seq = self.isequences[i]
        one_hot = (seq[..., np.newaxis] == np.arange(len(self.alphabet))).astype('float32')
        return torch.from_numpy(one_hot), torch.from_numpy(seq)

Let's try it out on our sequences. Remember what they looked like?

In [ ]:
sequences

Ok, here goes:

In [ ]:
seq_dataset = SeqDataset(sequences)

The dataset object permits indexing (in general just with integers, but the way we implemented, it also works with slices)

In [ ]:
sequences[10]

In [ ]:
seq_dataset[10]

In [ ]:
seq_dataset[10:12]

This dataset object outputs our input sequences one-hot encoded and our target indices as tuples. This is the perfect object to hand to `torch.utils.data.DataLoader`, which can collect this into batches

In [ ]:
data = torch.utils.data.DataLoader(seq_dataset, batch_size=32)

Let's check out one of these batches:

In [ ]:
for x, y in data:
    break # stop loop after one iteration, just to fill x and y once

In [ ]:
x.shape, y.shape

As expected, it handed us input and output for 32 sequences of length 10.

##### Create a loss and an optimizer and do some training
We create the usual objects for training a neural network. 

- The network
- an optimization criterion
- an optimizer

We already have the train data loader.

In [ ]:
seq_encoder_decoder = SeqEncoderDecoder(len(alphabet), 20)

Recall that the loss function needs to swap some axes:

In [ ]:
criterion = lambda x, y: torch.nn.CrossEntropyLoss()(x.permute(0, 2, 1), y)

A good first-try optimizer is Adam:

In [ ]:
optimizer = torch.optim.Adam(seq_encoder_decoder.parameters())

Let's train this for some epochs!:

In [ ]:
all_losses = []
for i in range(100):
    losses = train_epoch(seq_encoder_decoder, data, criterion, optimizer)
    all_losses.append(losses)
all_losses = np.array(all_losses)

In [ ]:
plt.figure(figsize=(20, 2))
plt.plot(np.concatenate(all_losses))


Judging by the training loss decrease, it looks like the neural network "gets it"!

**Suggestion:** Try to look at the plot with the y-axis in log scale.

##### Check this on some sequences
We'll now generate some test sequences by hand and see the predictions of our network

First we generate some random test sequences of length `sequence_length` from our alphabet (we could check whether they are already in the train set or not, but at least some of them won't be, and it's hard for a model so small to memorize all train sequences and be bad only on test sequences). 

In [ ]:
test_sequences = np.random.choice(alphabet, (100, sequence_length))
test_sequences[:3]

Next we create the one-hot-encoded versions of the sequences

In [ ]:
test_input = torch.from_numpy((test_sequences[..., np.newaxis] == alphabet).astype('float32'))

We run the test sequences through our encoder-decoder

In [ ]:
test_output = seq_encoder_decoder(test_input).detach().cpu().numpy()

In order to read the sequences in their original alphabet, we convert them in two steps: First we find the strongest-activated dimension in the alphabet axis, and then we look up the corresponding letter:

In [ ]:
test_output_sequences = alphabet[test_output.argmax(axis=2)]

Let's take a look at the output!

In [ ]:
test_output_sequences[:3]

Does it correspond? Let's check!

In [ ]:
(test_output_sequences == test_sequences).mean()

Looks like the output corresponds perfectly to the input sequences!

###### Exercise 02:
**1\.** Make it learn to output the sequence backwards. You can do this by creating a `SeqDataset` object that has a flag indicating whether the output should be reversed, and have it feed the reversed prediction target in.
(Hint: Just copy and paste most of the above and make a little modification to `SeqDataset`)

In [ ]:
# %load solutions/rnn/solution02.1.txt
# %load https://raw.githubusercontent.com/SFdS-atelier-3/block-3/master/solutions/rnn/solution02.1.txt

2\. Make a bigger alphabet, vary sequence length at training. See how many hidden units you need to accommodate.

*This takes some for loops and will probably be to time-consuming for the class, so we will probably skip it*

3\. Feed a sequence of different length through an RNN trained for length 10. Evaluate the encoding/decoding performance for sequences of length 2 to 20

In [ ]:
# %load solutions/rnn/solution02.3.txt
# %load https://raw.githubusercontent.com/SFdS-atelier-3/block-3/master/solutions/rnn/solution02.3.txt

Huh, well that is awkward! The error is zero only at exactly the sequence length we trained the RNN with. Looks like our instructions were very precise ("learn to copy sequences of length 10") and followed *very precisely*...

RNNs develop their power in treating sequences of arbitrary length, so it would be really nice if our example here could benefit from that. In the process of fixing this we will learn about an important concept: extra symbols for end of sequence.

(But: What happens if we just train this thing on different-length sequences? Will it forget stuff?)

##### Introducing "punctuation" for RNNs

RNNs are powerful sequence processors, but we need to make sure to be very precise on how we train them. The above example shows that we cannot expect an RNN that we have trained to copy or reverse sequences of length 10 will be able to do so for any other number except 10. This is surprising at first, but when one accepts that RNNs just try to minimize their losses, one can imagine that it might do the first best thing, not them "human generalized" thing we had in mind.

In order to train variable-length sequences, we can add symbols to our sequences that tell the network to stop iterating.

The concept is easily explained: We add symbols to our sequences which signify certain operations. Right now we need to indicate "end of sequence". Let's reserve 0 for that for all sequences

There are two levels of complexity to implementing this

1. One could just decide on what the longest sequence length should be, and pad all shorter sequences with the stop symbol. This will probably alleviate the problem above a bit.
2. It becomes quite apparent though that sequences with too much stop symbol in them might degrade even before reaching the end latent state. Also the output should not have the burden to output lots of stop symbols. Enter loss masks for the output, and selecting the last hidden state before stop symbol of the input.

We will check out both of these

Let's start by creating an alphabet, and generating a list of random integer sequence lengths, from which we then generate a list of different-length random sequences of alphabet symbols:

In [ ]:
alphabet = np.array(list('abc'))
n_sequences = 10000
sequence_lengths = np.random.randint(3, 15, (n_sequences,))
sequences = [np.random.choice(alphabet, sequence_length) for sequence_length in sequence_lengths]
sequence_lengths.sum(), sequence_lengths.mean(), sequences[:3]

The above numbers show: total number of symbols, average number of symbols per sequence, and three different sequences.

Let's create the alphabet/integer codes pair on all the sequences (by concatenating them)

In [ ]:
u_alphabet, i_sequences = np.unique(np.concatenate(sequences), return_inverse=True)
u_alphabet, i_sequences.shape

Now that our sequence lengths are different for each sample, we need to place them into a 2D array which can accommodate the maximum sequence, and pad all the others with zeros. 

In order to figure out where to place the values, let's first make a mask using some smart broadcasting:

In [ ]:
length_mask = sequence_lengths[:, np.newaxis] > np.arange(max(sequence_lengths))

For each sequence length in the column vector `sequence_lengths[:, np.newaxis]` we obtain a boolean vector in which the first n entries are True, where n in the current sequence length, and the remaining entries are False.

Its shape is `(number of sequences, maximum sequence length)`

In [ ]:
length_mask.shape

To get a feeling for what this looks like, we can display it as an image:

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(length_mask, aspect='auto')

Now we can use this mask to place our `isequences` into a square array. 
All we have to do is use *array masking* to place the sequences at the appropriate locations. If we add 1 to `i_sequences`, then our non-stop-symbols are `1, 2, 3` and our stop-symbol can be 0.

We first make an integer array of the same size as our mask:

In [ ]:
padded_sequences = np.zeros_like(length_mask, dtype='int')

Then we assign the sequences (indices shifted by 1) to this square array

In [ ]:
padded_sequences[length_mask] = i_sequences + 1

Let's take a look at these padded sequences

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(padded_sequences, aspect='auto')

Let's train an *encoder-decoder* copying RNN with that, exactly like before

Most of our procedure carries over immediately, since the sequences we input are not very much different from the ones before, except that there is a strong prevalence for zeros at the end of them.

We create our `SeqDataset` object 

In [ ]:
var_seq_dataset = SeqDataset(padded_sequences)

We create a data loader

In [ ]:
var_data = torch.utils.data.DataLoader(var_seq_dataset, batch_size=32, shuffle=True)

And a network that takes input size one bigger than the alphabet length in order to be able to accommodate the stop symbol

In [ ]:
var_seq_encoder_decoder = SeqEncoderDecoder(len(alphabet) + 1, 200)

As before, we need an optimization criterion

In [ ]:
criterion = lambda x, y: torch.nn.CrossEntropyLoss()(x.permute(0, 2, 1), y)

And we need an optimizer

In [ ]:
optimizer = torch.optim.Adam(var_seq_encoder_decoder.parameters())

Let's train that network for a bit

In [ ]:
var_losses = []

In [ ]:
for i in range(20):  # 200 does work better
    var_losses.append(train_epoch(var_seq_encoder_decoder, var_data, criterion, optimizer))

And let's take a look at how well the network trained this

In [ ]:
plt.plot(np.concatenate(var_losses))

That looks much much worse! What is going on?

Let's get a visual of how the predictions look by iterating through the train set

In [ ]:
train_output = []
for x, y in var_seq_dataset:
    train_output.append(var_seq_encoder_decoder(x[np.newaxis]).detach().cpu().numpy().argmax(2))

We can plot the output as an image of sequences

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(np.array(train_output)[:, 0], aspect='auto')

In order to visually assess the quality of these predictions, we can check whether they correspond with the input

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(np.array(train_output)[:, 0] == var_seq_dataset.sequences, aspect='auto')

We observe that they correspond at the beginning (not always, depending on training) and at the end. The end is mostly the padding with the stop symbol, which means that the RNN guessed the sequence length right. Since it gets rewarded equally for each one of these predictions, predicting a constant slab of stop symbols already gets you pretty far. Getting the first few right also helps towards the prediction goal.


Let's evaluate the accuracy of the predictions outside the stop symbols

In [ ]:
(np.array(train_output)[:, 0] == var_seq_dataset.sequences)[length_mask].mean()

Not perfect, but not completely random either.

If we included all the stop symbols, then our predictive accuracy would be

In [ ]:
(np.array(train_output)[:, 0] == var_seq_dataset.sequences).mean()

Let's generate some actual predicted symbol sequences. Here is our ordered alphabet:

In [ ]:
u_alphabet

We add an element to it: the empty string. By placing it at index 0, it will catch all stop symbols and they won't appear when the prediction is concatenated

In [ ]:
modified_alphabet = np.concatenate([[''], u_alphabet])

We loop through the sequences, and show the true sequence on the left and the predicted sequence on the right.

In [ ]:
for l, m in zip(modified_alphabet[var_seq_dataset.sequences], 
                modified_alphabet[np.array(train_output)[:, 0]]):
    print("".join(l), len("".join(l)), "-", "".join(m), len("".join(m)))

It seems to have gotten the length right very consistently, but is making errors with the actual symbols. The content is not completely wrong, but could be better.

We can speculate that the RNN was just too occupied to get the 0s at the end correctly predicted.

Making the network bigger and giving it more epochs to train shows that it does get better at memorizing the sequence, but also that the loss sometimes becomes very big again and needs to decrease from the beginning (probably due to strange loss shape).

##### Using the built-in RNN module
Before we move on to addressing these issues,  let's introduce something helpful -- a built-in `pytorch` RNN module, that can shorten our code.

Let's take a moment now to explore what `pytorch` can offer us in terms of pre-made objects.

Once an understanding is acquired, using pre-made objects for RNNs has multiple advantages:
- they permit writing less code
- other people can understand it better
- they might be optimized and run faster

Let's explore the object `torch.nn.RNN`

In [ ]:
#rnn = torch.nn.RNN?

In [ ]:
rnn = torch.nn.RNN

In [ ]:
rnn = torch.nn.RNN(input_size=4, hidden_size=10, batch_first=True, num_layers=1)

In [ ]:
out = rnn.forward(torch.randn(2, 5, 4))

In [ ]:
out[0].shape, out[1].shape

By matching shapes we conclude that the first output is the hidden state. According to the docstring the second output is the last hidden state. Let's check:

In [ ]:
out[0][:, -1] - out[1][-1]

Why report both of these?

It starts making a difference when we set `num_layers > 1`. Then the last state will be given across all layers, but the rest will only give the output of the last layer.

OK, let's add a 1-layer `RNN` object into our encoder and decoder classes:

In [ ]:
class SeqEncoderRNN(torch.nn.Module):
    def __init__(self, n_input, n_hidden):
        super(SeqEncoderRNN, self).__init__()
        self.n_input = n_input
        self.n_hidden = n_hidden
        
        self.build()
    
    def build(self):
        self.rnn = torch.nn.RNN(input_size=self.n_input,
                                hidden_size=self.n_hidden,
                                num_layers=1, batch_first=True)
    
    def forward(self, x):
        # assume x.shape = batch, time, features
        out, hidden = self.rnn(x)
        return hidden[0]

In [ ]:
class SeqDecoderRNN(torch.nn.Module):
    def __init__(self, n_hidden, n_output):
        super(SeqDecoderRNN, self).__init__()
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        self.build()
    
    def build(self):
        self.rnn = torch.nn.RNN(input_size=1, hidden_size=self.n_hidden, batch_first=True)
        self.hidden_to_output = torch.nn.Linear(self.n_hidden, self.n_output)
    
    def forward(self, initial_hidden, n):
        batch_size, n_hidden = initial_hidden.shape
        
        # input a bunch of zeros, because the RNN requires input
        inputs = torch.zeros(batch_size, n, 1, device=initial_hidden.device)
        outputs, hidden = self.rnn(inputs, initial_hidden[np.newaxis])
        
        # outputs will be of size batch, time, n_hidden
        # need to linear transform them to output size
        
        out = self.hidden_to_output(outputs.reshape(batch_size * n, self.n_hidden)
                                   ).reshape(batch_size, n, self.n_output)
        
        return out
        

In [ ]:
class SeqEncoderDecoderRNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SeqEncoderDecoderRNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        self.build()
    
    def build(self):
        self.encoder = SeqEncoderRNN(self.input_dim, self.hidden_dim)
        self.decoder = SeqDecoderRNN(self.hidden_dim, self.input_dim)
    
    def forward(self, x):
        _, time_length, _ = x.shape
        hidden = self.encoder(x)
        output = self.decoder(hidden, time_length)
        return output

That looks a bit shorter than before.

Another step to making this shorter would be to integrate everything into `SeqEncoderDecoderRNN`

Let's train this to see that we get similar results as before, and then move on to masking

In [ ]:
var_seq_encoder_decoder_rnn = SeqEncoderDecoderRNN(len(alphabet) + 1, 200)

In [ ]:
criterion = lambda x, y: torch.nn.CrossEntropyLoss()(x.permute(0, 2, 1), y)
optimizer = torch.optim.Adam(var_seq_encoder_decoder_rnn.parameters())

In [ ]:
var_losses = []
import sys

In [ ]:
for i in range(20): #100
    var_losses.append(train_epoch(var_seq_encoder_decoder_rnn, var_data, criterion, optimizer))
    print(".", end="")
    sys.stdout.flush()

In [ ]:
plt.plot(np.concatenate(var_losses))

In [ ]:
train_output = []
for x, y in var_seq_dataset:
    train_output.append(var_seq_encoder_decoder_rnn(x[np.newaxis]).detach().cpu().numpy().argmax(2))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(np.array(train_output)[:, 0], aspect='auto')

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(np.array(train_output)[:, 0] - var_seq_dataset.sequences, aspect='auto')

In [ ]:
(np.array(train_output)[:, 0] == var_seq_dataset.sequences)[length_mask].mean()

In [ ]:
var_seq_dataset.sequences

In [ ]:
u_alphabet

In [ ]:
modified_alphabet = np.concatenate([[''], u_alphabet])

In [ ]:
for l, m in zip(modified_alphabet[var_seq_dataset.sequences], 
                modified_alphabet[np.array(train_output)[:, 0]]):
    print("".join(l), len("".join(l)), "-", "".join(m), len("".join(m)))

##### Using masks to restrict to region of interest

We now introduce a way of restricting the learning to the parts of the sequences that are not stop symbols. This will take us towards general and efficient ways of training RNNs.

Here we modify our architecture at two points:
1. at the encoder level, we do not wait until the end of the sequence to return the hidden state, but rather return the hidden state upon the first encounter of a stop symbol (i.e. when the sequence is considered over)
2. at the training level, we only care about the loss up to the first stop symbol in the target sequence

In [ ]:
class SeqEncoderRNNMasked(torch.nn.Module):
    def __init__(self, n_input, n_hidden):
        super(SeqEncoderRNNMasked, self).__init__()
        self.n_input = n_input
        self.n_hidden = n_hidden
        
        self.build()
    
    def build(self):
        self.rnn = torch.nn.RNN(input_size=self.n_input,
                                hidden_size=self.n_hidden,
                                num_layers=1, batch_first=True)
    
    def forward(self, x, seq_lengths):
        # assume x.shape = batch, time, features
        out, hidden = self.rnn(x)
        # take advantage of the fact that for 1-layer rnn
        # out gives you the hidden states
        return out[torch.arange(x.shape[0]), seq_lengths.reshape(-1)]


In [ ]:
class SeqEncoderDecoderRNNMasked(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SeqEncoderDecoderRNNMasked, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        self.build()
    
    def build(self):
        self.encoder = SeqEncoderRNNMasked(self.input_dim, self.hidden_dim)
        self.decoder = SeqDecoderRNN(self.hidden_dim, self.input_dim)
    
    def forward(self, x, seq_lengths):
        _, time_length, _ = x.shape
        
        hidden = self.encoder(x, seq_lengths)
        output = self.decoder(hidden, time_length)
        return output

In [ ]:
class VarSeqDataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
        
        self.build()
    
    def build(self):
        self.sequence_lengths = np.array(list(map(len, self.sequences)))
        self.alphabet, self.isequences = np.unique(np.concatenate(self.sequences), return_inverse=True)
        self.mask = self.sequence_lengths[:, np.newaxis] > np.arange(self.sequence_lengths.max() + 1)
        self.padded_sequences = np.zeros_like(self.mask, dtype='int')
        self.padded_sequences[self.mask] = self.isequences + 1
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, i):
        seq = self.padded_sequences[i]
        one_hot = (seq[..., np.newaxis] == np.arange(len(self.alphabet) + 1)).astype('float32')
        return torch.from_numpy(one_hot), torch.from_numpy(seq), torch.from_numpy(self.sequence_lengths[i:i+1])

In [ ]:
vsd = VarSeqDataset(sequences)

In [ ]:
vsd[10]

In [ ]:
def train_epoch_var(model, dataset, criterion, optimizer):
    losses = []
    for x, y, l in dataset:
        p = model(x, l)
        loss = criterion(p, y)
        losses.append(loss.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return np.array(losses)


In [ ]:
sedrm = SeqEncoderDecoderRNNMasked(len(alphabet) + 1, 200)

In [ ]:
optimizer = torch.optim.Adam(sedrm.parameters())

In [ ]:
var_len_data = torch.utils.data.DataLoader(vsd, batch_size=32, shuffle=True)

In [ ]:
var_len_losses = []

In [ ]:
for i in range(20): #100
    var_len_losses.append(train_epoch_var(sedrm, var_len_data, criterion, optimizer))
    print(".", end="")
    sys.stdout.flush()

In [ ]:
plt.plot(np.concatenate(var_len_losses))

In [ ]:
train_output = []
for x, y, l in vsd:
    train_output.append(sedrm(x[np.newaxis], l).detach().cpu().numpy().argmax(2))

In [ ]:
train_output = np.vstack(train_output)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(train_output - vsd.padded_sequences, aspect='auto')

In [ ]:
for l, m in zip(modified_alphabet[vsd.padded_sequences], 
                modified_alphabet[train_output]):
    print("".join(l), len("".join(l)), "-", "".join(m), len("".join(m)))

Now let's add in a training function that cuts the loss at the first stop symbol

In [ ]:
def train_epoch_var_smart(model, dataset, criterion, optimizer):
    losses = []
    for x, y, l in dataset:
        p = model(x, l)
        
        mask = l >= torch.arange(y.shape[1])
        
        loss = criterion(p[mask], y[mask])
        losses.append(loss.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return np.array(losses)


In [ ]:
sedrm2 = SeqEncoderDecoderRNNMasked(len(alphabet) + 1, 200)

In [ ]:
optimizer = torch.optim.Adam(sedrm2.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
var_len_data = torch.utils.data.DataLoader(vsd, batch_size=32, shuffle=True)

In [ ]:
var_len_losses = []

In [ ]:
for i in range(20): # 100
    var_len_losses.append(train_epoch_var_smart(sedrm2, var_len_data, criterion, optimizer))
    print(".", end="")
    sys.stdout.flush()

In [ ]:
plt.plot(np.concatenate(var_len_losses))

#### A LSTM one that can do additions

Now that we have gotten to grips with the plumbing of RNNs using the very simple copy task, let's apply our knowledge to an example that is slightly less trivial: Learning arithmetic, specifically addition, on text sequences


##### Let's first come up with a way of creating data

Let's come up with a way of generating many many integer addition examples

In [ ]:
from sklearn.utils import check_random_state
def generate_additions_x_plus_y(n_examples, min_len_x=3, max_len_x=4, min_len_y=3, max_len_y=4, random_state=0):
    # note that this creates most numbers of the max length
    rng = check_random_state(random_state)
    min_x = 10 ** (min_len_x - 1)
    max_x = 10 ** max_len_x - 1
    min_y = 10 ** (min_len_y - 1)
    max_y = 10 ** max_len_y - 1
    
    xs = rng.randint(min_x, max_x + 1, n_examples)
    ys = rng.randint(min_y, max_y + 1, n_examples)
    sums = [(f"{x}+{y}", f"{x + y}") for x, y in zip(xs, ys)]
    return sums

In [ ]:
sums, outputs = zip(*generate_additions_x_plus_y(20000))

In [ ]:
sums_dataset = VarSeqDataset(list(map(list, sums)))

In [ ]:
sums_dataset[361]

In [ ]:
outputs_dataset = VarSeqDataset(list(map(list, outputs)))

In [ ]:
class SumRNN(torch.nn.Module):
    
    def __init__(self, n_inputs, n_outputs, n_hidden, n_layers=1):
        super(SumRNN, self).__init__()
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        
        self.build()
    
    def build(self):
        self.encoder_rnn = torch.nn.RNN(input_size=self.n_inputs,
                                        hidden_size=self.n_hidden,
                                        num_layers=self.n_layers,
                                        batch_first=True)
        self.decoder_rnn = torch.nn.RNN(input_size=1,
                                        hidden_size=self.n_hidden,
                                        num_layers = self.n_layers,
                                        batch_first=True)
        self.hidden_to_output = torch.nn.Linear(self.n_hidden,
                                                self.n_outputs)
    
    def encode(self, x):
        out, hidden = self.encoder_rnn(x)
        return hidden
    
    def decode(self, initial_hidden, n):
        n_layers, batch_size, n_hidden = initial_hidden.shape
        
        zero_input = torch.zeros(batch_size, n, 1, device=initial_hidden.device)
        out, hidden = self.decoder_rnn(zero_input, initial_hidden)
        
        output = self.hidden_to_output(
            out.reshape(batch_size * n, n_hidden)).reshape(batch_size, n, self.n_outputs)
        return output

    def forward(self, x):
        encoded = self.encode(x)
        decoded = self.decode(encoded, x.shape[1])
        return decoded

        

In [ ]:
sums_outputs = ZipDataset(sums_dataset, outputs_dataset)
sums_dataloader = torch.utils.data.DataLoader(sums_outputs, batch_size=32, shuffle=True)

In [ ]:
sum_rnn = SumRNN(len(sums_dataset.alphabet) + 1, len(outputs_dataset.alphabet) + 1, 100, n_layers=5)

In [ ]:
criterion = lambda x,y: torch.nn.CrossEntropyLoss()(x.permute(0, 2, 1), y)
optimizer = torch.optim.Adam(sum_rnn.parameters())

In [ ]:
def train_epoch1(model, dataset, criterion, optimizer):
    losses = []
    for x, y in dataset:
        p = model(x[0])
        loss = criterion(p[:, :y[0].shape[1]], y[1])
        losses.append(loss.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return np.array(losses)

In [ ]:
all_sum_losses = []

In [ ]:
import time

In [ ]:
t0 = time.time()
for i in range(20): # 20 epochs takes 10min on laptop
    losses = train_epoch1(sum_rnn, sums_dataloader, criterion, optimizer)
    all_sum_losses.append(losses)
    print(".", end="")
    sys.stdout.flush()
t1 = time.time()
print(f"\nThis took {t1-t0}s")

In [ ]:
plt.plot(np.concatenate(all_sum_losses))

In [ ]:
with torch.no_grad():
    predictions = [sum_rnn(x[0][np.newaxis]).detach().cpu().numpy().argmax(-1) for x in sums_dataset]

In [ ]:
predictions = np.vstack(predictions)

In [ ]:
output_alphabet_aug = np.concatenate([[''], outputs_dataset.alphabet])

In [ ]:
predictions_symbols = output_alphabet_aug[predictions]
["".join(line) for line in predictions_symbols]

In [ ]:
for s, o, p in zip(sums, outputs, predictions_symbols):
    print(s, "=", o, "?", "".join(p))

Finally, let's replace the simple RNN by an LSTM!

In [ ]:
class SumLSTM(torch.nn.Module):
    
    def __init__(self, n_inputs, n_outputs, n_hidden, n_layers=1):
        super(SumLSTM, self).__init__()
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        
        self.build()
    
    def build(self):
        self.encoder_rnn = torch.nn.LSTM(input_size=self.n_inputs,
                                        hidden_size=self.n_hidden,
                                        num_layers=self.n_layers,
                                        batch_first=True)
        self.decoder_rnn = torch.nn.LSTM(input_size=1,
                                        hidden_size=self.n_hidden,
                                        num_layers = self.n_layers,
                                        batch_first=True)
        self.hidden_to_output = torch.nn.Linear(self.n_hidden,
                                                self.n_outputs)
    
    def encode(self, x):
        out, hidden = self.encoder_rnn(x)
        return hidden
    
    def decode(self, initial_hidden, n):
        n_layers, batch_size, n_hidden = initial_hidden[0].shape
        
        zero_input = torch.zeros(batch_size, n, 1, device=initial_hidden[0].device)
        out, hidden = self.decoder_rnn(zero_input, initial_hidden)
        
        output = self.hidden_to_output(
            out.reshape(batch_size * n, n_hidden)).reshape(batch_size, n, self.n_outputs)
        return output

    def forward(self, x):
        encoded = self.encode(x)
        decoded = self.decode(encoded, x.shape[1])
        return decoded


In [ ]:
sum_lstm = SumLSTM(len(sums_dataset.alphabet) + 1, len(outputs_dataset.alphabet) + 1, 200, n_layers=5)

In [ ]:
criterion = lambda x,y: torch.nn.CrossEntropyLoss()(x.permute(0, 2, 1), y)
optimizer = torch.optim.Adam(sum_lstm.parameters())

In [ ]:
def train_epoch1(model, dataset, criterion, optimizer):
    losses = []
    for x, y in dataset:
        p = model(x[0])
        loss = criterion(p[:, :y[0].shape[1]], y[1])
        losses.append(loss.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return np.array(losses)

In [ ]:
all_sum_losses = []

In [ ]:
import time

In [ ]:
t0 = time.time()
for i in range(10):
    losses = train_epoch1(sum_lstm, sums_dataloader, criterion, optimizer)
    all_sum_losses.append(losses)
    print(".", end="")
    sys.stdout.flush()
t1 = time.time()
print(f"\nThis took {t1-t0}s")

In [ ]:
plt.plot(np.concatenate(all_sum_losses))

In [ ]:
with torch.no_grad():
    predictions = [sum_lstm(x[0][np.newaxis]).detach().cpu().numpy().argmax(-1) for x in sums_dataset]

In [ ]:
predictions = np.vstack(predictions)

In [ ]:
predictions_symbols = output_alphabet_aug[predictions]

In [ ]:
for s, o, p in zip(sums, outputs, predictions_symbols):
    print(s, "=", o, "?", "".join(p))

### Bigger RNNs
Now let's move on to some real-life examples
#### A character-level predictive RNN
Following Andrej Karpathy's famous blog post, we will implement an RNN that can predict the next character in a string. For this, let's first find an appropriate text.

Let's download something from the Gutenberg project:

In [ ]:
!if [ ! -f ./moliere1.txt ]; then wget http://www.gutenberg.org/files/40086/40086-0.txt -O ./moliere1.txt; fi

In [ ]:
import os
if not os.path.exists("./moliere.txt"):
    import urllib.request
    urllib.request.urlretrieve("http://www.gutenberg.org/files/40086/40086-0.txt", "./moliere1.txt")

And now let's take a look at the beginning of the text:

In [ ]:
with open("./moliere1.txt", "rb") as f:
    content = f.read().decode()
content = content[31500:]

In [ ]:
print(content[:2000])

A glimpse at the unique characters in this text:

In [ ]:
alphabet, icontent = np.unique(list(content), return_inverse=True)
alphabet

In [ ]:
counts = np.bincount(icontent)
plt.figure(figsize=(20, 2))
plt.bar(np.arange(len(counts)), counts)
plt.xticks(np.arange(len(counts)), alphabet)
pass

We now proceed to implement a neural network exactly like our sine-wave predictor, but for characters. We will use the LSTM out of the box in order to have as much representative power as possible.

First we need a dataset that turns our text as subsequences of a length we specify:

In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, text, seq_length=100):
        self.text = text
        self.seq_length = seq_length
        
        self.build()
    
    def build(self):
        self.alphabet, self.itext = np.unique(list(self.text), return_inverse=True)
    
    def __len__(self):
        return len(self.text) - self.seq_length + 1
    
    def __getitem__(self, i):
        return self.itext[i:i + self.seq_length] + 1

Let's implement another dataset that takes a text dataset and makes it return two copies of each sequence, shifted by 1:

In [ ]:
class ShiftByOne(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset) - 1
    
    def __getitem__(self, i):
        return self.dataset[i], self.dataset[i + 1]


In [ ]:
text_data = TextDataset(content)

In [ ]:
shifted_text_data = ShiftByOne(text_data)

In [ ]:
text_dataloader = torch.utils.data.DataLoader(shifted_text_data, batch_size=32, shuffle=True)

In [ ]:
x, y = next(iter(text_dataloader))

In [ ]:
text_data.alphabet[x.numpy() - 1], text_data.alphabet[y.numpy() - 1]

Instead of performing one-hot encoding, we will work with a direct embedding. This can be seen as an extra linear transform on top of the one-hot-encoded input. It permits us to adjust input dimensionality for large vocabularies

In [ ]:
class PredictorLSTM(torch.nn.Module):
    def __init__(self, n_symbols, n_hidden, n_layers=1, embedding_size=None):
        super(PredictorLSTM, self).__init__()
        self.n_symbols = n_symbols
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.embedding_size = embedding_size
        
        self.build()
    
    def build(self):
        if self.embedding_size is None:
            embedding_size = self.n_symbols
        else:
            embedding_size = self.embedding_size
        self.embeddings = torch.nn.Parameter(torch.randn(self.n_symbols, embedding_size) * 0.01)
        self.rnn = torch.nn.LSTM(input_size=embedding_size, hidden_size=self.n_hidden,
                                batch_first=True, num_layers=self.n_layers)
        self.hidden_to_output = torch.nn.Linear(self.n_hidden, self.n_symbols)
    
    def forward(self, x):
        embedded = self.embeddings[x]
        out, cell_hidden = self.rnn(embedded)
        output = self.hidden_to_output(out.reshape(-1, self.n_hidden)).reshape(out.shape[:2] + (self.n_symbols,))
        return output
        

In [ ]:
plstm = PredictorLSTM(n_symbols=len(text_data.alphabet) + 1, n_hidden=512, embedding_size=64, n_layers=2)

In [ ]:
criterion = lambda x, y: torch.nn.CrossEntropyLoss()(x.permute(0, 2, 1), y)

In [ ]:
optimizer = torch.optim.Adam(plstm.parameters())

In [ ]:
p = plstm.forward(x)

In [ ]:
p.shape

In [ ]:
criterion(p, y)

In [ ]:
def train_epoch(model, dataset, criterion, optimizer, max_batches=None):
    losses = []
    if max_batches is None:
        max_batches = len(dataset)
    for i, (x, y) in zip(range(max_batches), dataset):
        p = model(x)
        loss = criterion(p, y)
        losses.append(loss.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return np.array(losses)

In [ ]:
all_losses = []

In [ ]:
len(all_losses)

In [ ]:
%timeit all_losses.append(train_epoch(plstm, text_dataloader, criterion, optimizer, max_batches=1000))

In [ ]:
plt.plot(np.concatenate(all_losses))

OK, now for predictions.

If you remember the way we did this for the sine wave: We put in some initial state, push it through the network and get an output. This output we feed in to obtain the next step.

The pytorch LSTM module does not allow super easy prediction that way, so we have to do it ourselves. One way of doing this is to feed an increasingly larger array, adding on each prediction as input.

In [ ]:
initial_letters = torch.arange(35, 36)

In [ ]:
prediction_array = torch.zeros((len(initial_letters), 1000), dtype=torch.long)
prediction_array[:, 0] = initial_letters

In [ ]:
for i in range(1, prediction_array.shape[1]):
    pred_logits = plstm.forward(prediction_array[:, :i])[:, -1, :].detach()
    pred_prob = torch.nn.Softmax(dim=1)(pred_logits)
    samples = torch.multinomial(pred_prob, 1)
    prediction_array[:, i] = samples[:, 0]

In [ ]:
[print("".join(row)) for row in text_data.alphabet[prediction_array.numpy() - 1]]

##### But also a dictionary-based Markov model

As a response to Andrej Karpathy's blog post about *The Unreasonable Effectiveness of Recurrent Neural Networks*, Yoav Goldberg showed that a very simple N-back Markov model is an extremely impressive baseline.

The idea is to learn a sparse conditional probability distribution of a letter given N previous letters.

We can easily learn this with a dictionary of lists:

In [ ]:
def create_markov_dict(text, markov_order=4):
    
    unique_characters = np.unique(list(text))
    n_unique = len(unique_characters)
    transl = dict(list(zip(unique_characters, np.arange(n_unique))) +
                  list(zip(np.arange(n_unique), unique_characters)))
    markov_dict = dict()
    for i in range(len(text) - markov_order - 1):
        key = text[i:i + markov_order]
        value = text[i + markov_order]
        freq_table = markov_dict[key] = markov_dict.get(key, np.zeros(n_unique, dtype='int'))
        freq_table[transl[value]] += 1
    return markov_dict, transl

In [ ]:
d, t = create_markov_dict(content, markov_order=6)

In [ ]:
len(d)

We can generate text from this dictionary by starting off with some characters that appear in the dictionary, and randomly sampling from the possibilities of the next letter:

In [ ]:
def generate_from_markov(markov_dict, transl_dict, n=1000, starting_point=None):
    
    n_states = len(transl_dict) // 2
    if starting_point is None:
        starting_point = np.random.choice(list(markov_dict.keys()))
    
    output = list(starting_point)
    cur_key = starting_point
    
    for i in range(n):
        counts = markov_dict[cur_key]
        counts = counts / counts.sum()
        next_char_ind = np.random.multinomial(1, counts).argmax()
        next_char = transl_dict[next_char_ind]
        output.append(next_char)
        
        cur_key = cur_key[1:] + next_char
    return "".join(output)

In [ ]:
generated = generate_from_markov(d, t)

In [ ]:
print(generated)